In [198]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from sklearn.metrics import  roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE


In [220]:
df=pd.read_csv('diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [221]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [222]:
to_drop=df[(df.BMI==0 )&  (df.BloodPressure==0)]
to_drop.index

Index([49, 60, 81, 426, 494, 522, 706], dtype='int64')

In [223]:
df.drop(to_drop.index,inplace=True)
df.shape

(761, 9)

In [227]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape

(608, 8)

In [228]:
print(np.unique(y_train,return_counts=True))

(array([0, 1]), array([395, 213]))


In [229]:
smote = SMOTE(random_state=42)
x_train_bal, y_train_bal = smote.fit_resample(x_train, y_train)
print(np.unique(y_train_bal,return_counts=True))

(array([0, 1]), array([395, 395]))


In [232]:
sc=StandardScaler()
x_train_bal_sc=sc.fit_transform(x_train_bal)
x_test_sc=sc.transform(x_test)

In [233]:
lr=LogisticRegression(random_state=42)
lr.fit(x_train_bal_sc,y_train_bal)
y_prob_lr = lr.predict_proba(x_test_sc)[:, 1]  # Only take the probability of class 1
lr.score(x_train_bal_sc,y_train_bal),lr.score(x_test_sc,y_test)

(0.7544303797468355, 0.7647058823529411)

In [234]:
svc=SVC(gamma=0.01,random_state=42,probability=True)
svc.fit(x_train_bal_sc,y_train_bal)
y_prob_svc = svc.predict_proba(x_test_sc)[:, 1]
svc.score(x_train_bal_sc,y_train_bal),svc.score(x_test_sc,y_test)

(0.7582278481012659, 0.7647058823529411)

In [235]:
dt=DecisionTreeClassifier(max_depth=4,min_samples_leaf=40)
dt.fit(x_train_bal,y_train_bal)
y_prob_dt = dt.predict_proba(x_test)[:, 1]
dt.score(x_train_bal,y_train_bal),dt.score(x_test,y_test)

(0.7924050632911392, 0.7320261437908496)

In [236]:
rf=RandomForestClassifier(max_depth=7,min_samples_leaf=25,random_state=42)
rf.fit(x_train_bal,y_train_bal)
y_prob_rf = rf.predict_proba(x_test)[:, 1]
rf.score(x_train_bal,y_train_bal),rf.score(x_test,y_test)

(0.830379746835443, 0.7254901960784313)

In [237]:
score_lr=roc_auc_score(y_test,y_prob_lr)
score_svc=roc_auc_score(y_test,y_prob_svc)
score_dt=roc_auc_score(y_test,y_prob_dt)
score_rf=roc_auc_score(y_test,y_prob_rf)
print(score_lr,score_svc,score_dt,score_rf)

0.8290310512532736 0.8340815563037786 0.7996632996632997 0.8121960344182567
